In [3]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [4]:
volume_dir = '/home/yuncong/CSHL_volumes/'
mesh_rootdir = create_if_not_exists('/home/yuncong/CSHL_meshes')

In [5]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [6]:
volume_landmark_names_unsided = ['12N', '5N', '6N', '7N', '7n', 'AP', 'Amb', 'LC',
                                 'LRt', 'Pn', 'R', 'RtTg', 'Tz', 'VLL', 'sp5']
linear_landmark_names_unsided = ['outerContour']

labels_unsided = volume_landmark_names_unsided + linear_landmark_names_unsided
labels_unsided_indices = dict((j, i+1) for i, j in enumerate(labels_unsided))  # BackG always 0

labelMap_unsidedToSided = {'12N': ['12N'],
                            '5N': ['5N_L', '5N_R'],
                            '6N': ['6N_L', '6N_R'],
                            '7N': ['7N_L', '7N_R'],
                            '7n': ['7n_L', '7n_R'],
                            'AP': ['AP'],
                            'Amb': ['Amb_L', 'Amb_R'],
                            'LC': ['LC_L', 'LC_R'],
                            'LRt': ['LRt_L', 'LRt_R'],
                            'Pn': ['Pn_L', 'Pn_R'],
                            'R': ['R_L', 'R_R'],
                            'RtTg': ['RtTg'],
                            'Tz': ['Tz_L', 'Tz_R'],
                            'VLL': ['VLL_L', 'VLL_R'],
                            'sp5': ['sp5'],
                           'outerContour': ['outerContour']}

labelMap_sidedToUnsided = {n: nu for nu, ns in labelMap_unsidedToSided.iteritems() for n in ns}

from itertools import chain
labels_sided = list(chain(*(labelMap_unsidedToSided[name_u] for name_u in labels_unsided)))
labels_sided_indices = dict((j, i+1) for i, j in enumerate(labels_sided)) # BackG always 0

In [7]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

In [8]:
atlasVolume_landmark_bboxes = {name: bbox_3d(atlas_volume == labels_sided_indices[name])
                           for name in available_labels_sided}

In [9]:
atlas_mesh_dir = create_if_not_exists(os.path.join(mesh_rootdir, 'atlas'))

for name_s in available_labels_sided:

    print name_s

    xmin, xmax, ymin, ymax, zmin, zmax = atlasVolume_landmark_bboxes[name_s]

    vol = (atlas_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)
    
    t = time.time()

    polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5
    
    save_mesh_stl(polydata, atlas_mesh_dir + "/atlasVolume_%(name)s_smoothed.stl" % {'name': name_s})

12N
area: 43842.74

marching cube: 0.18 seconds
compute surface area: 0.01 seconds
fill point array: 0.06 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.06 seconds
simplify 0 @ 47517: 0.44 seconds
simplify 1 @ 9505: 0.43 seconds
simplify 2 @ 1902: 0.11 seconds



5N_L

time: 1.24
marching cube: 0.09 seconds



area: 27910.95

...


VLL_R

time: 1.21
marching cube: 0.21 seconds



area: 44407.16

compute surface area: 0.01 seconds
fill point array: 0.06 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.06 seconds
simplify 0 @ 44314: 0.44 seconds
simplify 1 @ 8864: 0.41 seconds
simplify 2 @ 1774: 0.08 seconds


time: 1.21


In [10]:
# Load Atlas meshes

atlas_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/atlas/atlasVolume_%(name)s_smoothed.stl" % {'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlas_polydata_list[name_s] = polydata

In [ ]:
subject_localAdjusted_polydata_list_allStacks = {}

In [19]:
# stack = 'MD593'

for stack in ['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']:

    atlasAlignParams_dir = '/home/yuncong/CSHL_atlasAlignParams_atlas/'
    with open(atlasAlignParams_dir + '/%(stack)s/%(stack)s_3dAlignParams.txt' % {'stack': stack}, 'r') as f:
        lines = f.readlines()
        global_params = np.array(map(float, lines[0].strip().split()))
    #     atlas_xdim, atlas_ydim, atlas_zdim  = np.array(map(float, lines[1].strip().split()))
        atlas_centroid = np.array(map(float, lines[2].strip().split()))
        test_xdim, test_ydim, test_zdim = np.array(map(int, lines[3].strip().split()))
        test_centroid = np.array(map(float, lines[4].strip().split()))


    parameters_allLandmarks = {}
    atlas_centroid_allLandmarks = {}
    test_centroid_allLandmarks = {}

    for name in available_labels_sided:

        with open(atlasAlignParams_dir + '/%(stack)s/%(stack)s_%(name)s_transformUponAffineProjection.txt' % \
                            {'stack': stack, 'name': name}, 'r') as f:
            lines = f.readlines()
            params = np.array(map(float, lines[0].strip().split()))
            test_xdim, test_ydim, test_zdim = np.array(map(int, lines[1].strip().split()))
            atlas_c = np.array(map(float, lines[2].strip().split()))
            test_c = np.array(map(float, lines[3].strip().split()))

        parameters_allLandmarks[name] = params
        atlas_centroid_allLandmarks[name] = atlas_c
        test_centroid_allLandmarks[name] = test_c


    subject_localAdjusted_polydata_list = {}

    for name_s in available_labels_sided:

        fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(label)s_smoothed.stl" % {'stack':stack, 'label':name_s}

        if os.path.exists(fn):
            vertices, faces = load_mesh_stl(fn)

    #         vertices = transform_points_inverse(global_params, pts_prime=vertices, 
    #                                             c_prime=test_centroid_allLandmarks[name], 
    #                                             c=atlas_centroid_allLandmarks[name])

            vertices = transform_points_inverse(global_params, pts_prime=vertices, 
                                        c_prime=test_centroid, c=atlas_centroid)

            polydata = mesh_to_polydata(vertices, faces)

            subject_localAdjusted_polydata_list[name_s] = polydata
            
    subject_localAdjusted_polydata_list_allStacks[stack] = subject_localAdjusted_polydata_list

fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 seconds
fill cell array: 0.00 seconds
fill point array: 0.00 s

In [ ]:
# subject_atlasProjected_polydata_list = {}

# for name_s in available_labels_sided:

#     fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(label)s_smoothed.stl" % {'stack':stack, 'label':name_s}

#     if os.path.exists(fn):
#         vertices, faces = load_mesh_stl(fn)
#         vertices = transform_points_inverse(global_params, pts_prime=vertices, 
#                                             c_prime=test_centroid, c=atlas_centroid)
#         polydata = mesh_to_polydata(vertices, faces)
        
#         subject_atlasProjected_polydata_list[name_s] = polydata

In [15]:
ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)

for i, (name_s, polydata) in enumerate(atlas_polydata_list.iteritems()):
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
#     a.GetProperty().SetColor((1.,1.,1.))    
    a.GetProperty().SetOpacity(.1)

    ren1.AddActor(a)
        
for i, (name_s, polydata) in enumerate(subject_localAdjusted_polydata_list.iteritems()):

    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
    a.GetProperty().SetColor((1.,1.,1.))

    ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()

In [21]:
# visualize all stacks

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)

for i, (name_s, polydata) in enumerate(atlas_polydata_list.iteritems()):
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
    a.GetProperty().SetColor((1.,1.,1.))    
#     a.GetProperty().SetOpacity(.1)

    ren1.AddActor(a)

for stack_ind, (st, polydata_list) in enumerate(subject_localAdjusted_polydata_list_allStacks.iteritems()):

    for i, (name_s, polydata) in enumerate(polydata_list.iteritems()):

        m = vtk.vtkPolyDataMapper()
        m.SetInputData(polydata)

        a = vtk.vtkActor()
        a.SetMapper(m)
        a.GetProperty().SetRepresentationToWireframe()
        a.GetProperty().SetColor(colors.values()[stack_ind+1])
#         a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
#         a.GetProperty().SetColor((1.,1.,1.))
        a.GetProperty().SetOpacity(.3)
    
        ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()